In [1]:
import sys, subprocess
def ensure(pkgs): subprocess.check_call([sys.executable, "-m", "pip", "install", "--quiet", *pkgs])
ensure(["ultralytics", "roboflow", "albumentations", "kaggle", "yolo", "supervision"])

## Датасетик

| Критерий                             | Обоснование                                                                                                                                                               |
| ------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **Реальная практическая значимость** | Детектирование касок и присутствия людей помогает автоматически контролировать технику безопасности на строительных и производственных площадках, снижая риски травм. :contentReference[oaicite:0]{index=0} |
| **Разнообразие и баланс классов**    | 3 класса: “Helmet”, “Person” и “Head”; несколько объектов на кадре, вариативные ракурсы, фон и плотность людей. Задача — object detection с множественными экземплярами. :contentReference[oaicite:1]{index=1} |
| **Размер и доступность**             | ~5000 изображений (≈1.3 GB), свободная лицензия CC0, скачивается через Kaggle CLI командой `!kaggle datasets download -d andrewmvd/hard-hat-detection -p ./data -q`. :contentReference[oaicite:2]{index=2} |
| **Качество аннотаций**               | Границы объектов размечены вручную в формате PASCAL VOC (bbox), верифицированы автором набора. Подходит для обучения детекторов типа YOLO, Faster R-CNN и др. :contentReference[oaicite:3]{index=3} |

In [ ]:
!kaggle datasets download -d andrewmvd/hard-hat-detection -p ./data -q

In [ ]:
!unzip ./data/safety-helmet-detection.zip -d ./data/helmet

In [2]:
from ultralytics import YOLO
import yaml, os
import supervision as sv

In [2]:
import xml.etree.ElementTree as ET
import os

classes = ['head', 'helmet', 'person']
base = './data/helmet'
images_dir = os.path.join(base, 'images')
xml_dir    = os.path.join(base, 'annotations')

def convert_xml_to_yolo(xml_file: str, txt_file: str):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w, h = float(size.find('width').text), float(size.find('height').text)
    lines = []
    for obj in root.findall('object'):
        cls = obj.find('name').text
        if cls not in classes: continue
        cls_id = classes.index(cls)
        xmin = float(obj.find('bndbox/xmin').text)
        ymin = float(obj.find('bndbox/ymin').text)
        xmax = float(obj.find('bndbox/xmax').text)
        ymax = float(obj.find('bndbox/ymax').text)
        x_c = ((xmin + xmax) / 2) / w
        y_c = ((ymin + ymax) / 2) / h
        bw  = (xmax - xmin) / w
        bh  = (ymax - ymin) / h
        lines.append(f"{cls_id} {x_c:.6f} {y_c:.6f} {bw:.6f} {bh:.6f}")
    os.makedirs(os.path.dirname(txt_file), exist_ok=True)
    with open(txt_file, 'w') as f:
        f.write('\n'.join(lines))

In [3]:
import random, shutil

ratios = {'train':0.8, 'val':0.1, 'test':0.1}
random.seed(42)

all_imgs = [f for f in os.listdir(images_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))]
random.shuffle(all_imgs)
n = len(all_imgs)
train_end = int(n * ratios['train'])
val_end   = train_end + int(n * ratios['val'])

splits = {
    'train': all_imgs[:train_end],
    'val':   all_imgs[train_end:val_end],
    'test':  all_imgs[val_end:]
}

for split, files in splits.items():
    for img in files:
        name, _ = os.path.splitext(img)
        dst_img = os.path.join(images_dir, split, img)
        os.makedirs(os.path.dirname(dst_img), exist_ok=True)
        shutil.copy(os.path.join(images_dir, img), dst_img)
        xml_path = os.path.join(xml_dir, name + '.xml')
        txt_path = os.path.join(base, 'labels', split, name + '.txt')
        if os.path.exists(xml_path):
            convert_xml_to_yolo(xml_path, txt_path)

In [ ]:
import yaml
data = {
    'path': base,
    'train': 'images/train',
    'val':   'images/val',
    'test':  'images/test',
    'nc':    len(classes),
    'names': classes
}
with open(os.path.join(base, 'data.yaml'), 'w') as f:
    yaml.dump(data, f, sort_keys=False)

## Baseline

### Обыкновенный

`YOLO('yolov8n.pt')` Загружает предобученные веса YOLOv8-nano (~3 M параметров) для детектирования по архитектуре Ultralytics.


In [3]:
model = YOLO('yolov8n.pt')

model.train(
    data='./data/helmet/data.yaml',
    imgsz=520,
    batch=32,
    epochs=20,
    project='lab8',
    name='y8n_base',
    device=0,
    resume=False,
    save_period=10
)

Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/helmet/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=520, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8n_base12, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, p

train: Scanning /home/data/helmet/labels/train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 688.5±367.0 MB/s, size: 271.8 KB)


val: Scanning /home/data/helmet/labels/val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to lab8/y8n_base12/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 544 train, 544 val
Using 6 dataloader workers
Logging results to lab8/y8n_base12
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        500       2612      0.944      0.591      0.633      0.425



20 epochs completed in 0.153 hours.
Optimizer stripped from lab8/y8n_base12/weights/last.pt, 6.2MB
Optimizer stripped from lab8/y8n_base12/weights/best.pt, 6.2MB

Validating lab8/y8n_base12/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.76it/s]


                   all        500       2612      0.945      0.594      0.636      0.425
                  head        108        708      0.893      0.898       0.92      0.618
                helmet        445       1871      0.941      0.884      0.955      0.638
                person          8         33          1          0     0.0337     0.0187
Speed: 0.2ms preprocess, 0.7ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to lab8/y8n_base12


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6c402e28f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### Трансформер

In [4]:
from ultralytics import RTDETR
YOLO = RTDETR

model = YOLO('rtdetr-x.pt')
model.train(
    data='./data/helmet/data.yaml',
    imgsz=520,
    batch=10,
    epochs=20,
    project='lab8',
    name='rtdetr_base',
    device=0
)

Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=10, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/helmet/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=520, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=rtdetr-x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=rtdetr_base3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning /home/data/helmet/labels/train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 519.1±192.8 MB/s, size: 271.8 KB)


val: Scanning /home/data/helmet/labels/val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to lab8/rtdetr_base3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 193 weight(decay=0.0), 256 weight(decay=0.00046875), 276 bias(decay=0.0)
Image sizes 544 train, 544 val
Using 6 dataloader workers
Logging results to lab8/rtdetr_base3
Starting training for 20 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


...

  0%|          | 0/400 [00:00<?, ?it/s]UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
      20/20      9.41G     0.3448     0.4027    0.08189         48        544: 100%|██████████| 400/400 [02:54<00:00,  2.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:04<00:00,  5.20it/s]


                   all        500       2612      0.605      0.633      0.628      0.426

20 epochs completed in 1.024 hours.
Optimizer stripped from lab8/rtdetr_base3/weights/last.pt, 135.4MB
Optimizer stripped from lab8/rtdetr_base3/weights/best.pt, 135.4MB

Validating lab8/rtdetr_base3/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
rt-detr-x summary: 370 layers, 65,473,601 parameters, 0 gradients, 222.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 25/25 [00:06<00:00,  3.76it/s]


                   all        500       2612       0.94      0.606      0.633      0.428
                  head        108        708      0.881       0.91      0.918      0.625
                helmet        445       1871      0.939      0.907      0.949      0.642
                person          8         33          1          0      0.031     0.0185
Speed: 0.1ms preprocess, 4.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to lab8/rtdetr_base3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6c2e4994b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

## Улучшенные baseline

### Обычная

In [10]:
strong_aug = dict(
    hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=15,
    translate=0.1, scale=0.7, shear=0.0, perspective=0.0005,
    mosaic=1.0, mixup=0.2, flipud=0.0, fliplr=0.5,
)

In [14]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(
    data='./data/helmet/data.yaml',
    imgsz=520,
    batch=32,
    epochs=20,
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15,
    translate=0.1,
    scale=0.7,
    shear=0.0,
    perspective=0.0005,
    mosaic=1.0,
    mixup=0.2,
    flipud=0.0,
    fliplr=0.5,
    project='lab8',
    name='y8n_plus',
    device=0
)

Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/helmet/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=520, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=y8n_plus3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0005, plots=True, pose=12.0, p

train: Scanning /home/data/helmet/labels/train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1373.0±1256.2 MB/s, size: 271.8 KB)


val: Scanning /home/data/helmet/labels/val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


...

                   all        500       2612      0.939      0.594      0.629      0.401



20 epochs completed in 0.124 hours.
Optimizer stripped from lab8/y8n_plus3/weights/last.pt, 6.2MB
Optimizer stripped from lab8/y8n_plus3/weights/best.pt, 6.2MB

Validating lab8/y8n_plus3/weights/best.pt...
Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
Model summary (fused): 72 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:05<00:00,  1.47it/s]


                   all        500       2612      0.942      0.583      0.629      0.405
                  head        108        708      0.867      0.887      0.911      0.584
                helmet        445       1871      0.958      0.861      0.951      0.615
                person          8         33          1          0      0.027     0.0146
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to lab8/y8n_plus3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f6c9a61d030>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

### Трансформер

In [ ]:
from ultralytics import RTDETR
YOLO = RTDETR

model = YOLO('rtdetr-x.pt')
model.train(
    data='./data/helmet/data.yaml',
    imgsz=520,
    batch=24,
    epochs=20,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=15,
    translate=0.1,
    scale=0.7,
    shear=0.0,
    perspective=0.0005,
    mosaic=1.0,
    mixup=0.2,
    flipud=0.0,
    fliplr=0.5,
    project='lab8',
    name='yonas_plus',
    device=0
)

Ultralytics 8.3.130 🚀 Python-3.10.0 torch-2.3.1+cu121 CUDA:0 (NVIDIA A100-PCIE-40GB, 40442MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./data/helmet/data.yaml, degrees=15, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=520, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=rtdetr-x.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yonas_plus, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0005, plots=True, pose=12.0

train: Scanning /home/data/helmet/labels/train.cache... 4000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4000/4000 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 672.2±632.2 MB/s, size: 271.8 KB)


val: Scanning /home/data/helmet/labels/val.cache... 500 images, 0 backgrounds, 0 corrupt: 100%|██████████| 500/500 [00:00<?, ?it/s]


Plotting labels to lab8/yonas_plus/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001429, momentum=0.9) with parameter groups 193 weight(decay=0.0), 256 weight(decay=0.0005625000000000001), 276 bias(decay=0.0)
Image sizes 544 train, 544 val
Using 6 dataloader workers
Logging results to lab8/yonas_plus
Starting training for 20 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/167 [00:00<?, ?it/s]UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
       1/20      19.5G      1.004     0.9767     0.4188        212        544:  75%|███████▌  | 126/167 [01:24<00:26,  1.55it/s]